# 🤖 LangGraph 多 Agent 系统实战

本 Notebook 展示如何使用 LangGraph 构建复杂的多 Agent 协作系统。

## 📋 学习目标

- LangGraph 基础概念（State、Node、Edge）
- 构建状态机工作流
- 多 Agent 协作模式
- 条件路由和决策
- 实战案例：智能内容创作团队


## 📦 安装依赖


In [ ]:
%pip install -q langgraph langchain langchain-openai
%pip install -q sentence-transformers


## 🔧 配置 DeepSeek API


In [ ]:
import os
from typing import TypedDict, Annotated, Sequence
from langchain_openai import ChatOpenAI

print("🔧 配置 DeepSeek API")
print("="*60)
print("⚠️  请将下方的 'your-deepseek-api-key-here' 替换为你的 API Key")
print("\n📝 获取 API Key:")
print("1. 访问 https://platform.deepseek.com/")
print("2. 注册/登录账号")
print("3. 创建 API Key")
print("="*60)

# 配置 DeepSeek API
os.environ["OPENAI_API_KEY"] = "your-deepseek-api-key-here"
os.environ["OPENAI_API_BASE"] = "https://api.deepseek.com"

llm = ChatOpenAI(
    model="deepseek-chat",
    openai_api_base="https://api.deepseek.com",
    temperature=0.7
)

print("\n✅ DeepSeek 配置完成！")


## 一、LangGraph 基础概念

### 核心概念

```
State (状态)
   ↓
Node (节点/Agent)
   ↓
Edge (边/转换)
   ↓
Graph (整个工作流)
```

### 简单示例


In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict

# 1. 定义状态
class SimpleState(TypedDict):
    messages: list[str]
    counter: int

# 2. 定义节点函数
def step1(state: SimpleState):
    print("执行步骤 1")
    return {
        "messages": state["messages"] + ["步骤1完成"],
        "counter": state["counter"] + 1
    }

def step2(state: SimpleState):
    print("执行步骤 2")
    return {
        "messages": state["messages"] + ["步骤2完成"],
        "counter": state["counter"] + 1
    }

# 3. 构建图
workflow = StateGraph(SimpleState)
workflow.add_node("step1", step1)
workflow.add_node("step2", step2)

# 4. 添加边
workflow.set_entry_point("step1")
workflow.add_edge("step1", "step2")
workflow.add_edge("step2", END)

# 5. 编译并运行
app = workflow.compile()

result = app.invoke({"messages": [], "counter": 0})
print("\n结果:")
print(f"消息: {result['messages']}")
print(f"计数: {result['counter']}")


### 2.1 定义状态


In [ ]:
from typing import TypedDict, List
import operator
from langchain.schema import HumanMessage, AIMessage

class AgentState(TypedDict):
    topic: str
    research_notes: str
    draft_content: str
    final_content: str
    messages: Annotated[List, operator.add]
    next_agent: str
    iteration: int
    max_iterations: int


### 2.2 创建各个 Agent


In [ ]:
from langchain.prompts import ChatPromptTemplate

# 研究员 Agent
def researcher_agent(state: AgentState) -> AgentState:
    print("\n📚 研究员 Agent 工作中...")
    
    prompt = ChatPromptTemplate.from_template(
        """你是一个专业的研究员。请针对以下主题进行研究，提供关键要点和有价值的信息。

主题：{topic}

请提供：
1. 核心概念定义
2. 重要特点（3-5个）
3. 实际应用场景
4. 注意事项

研究笔记："""
    )
    
    chain = prompt | llm
    result = chain.invoke({"topic": state["topic"]})
    
    return {
        **state,
        "research_notes": result.content,
        "messages": [f"[研究员] 完成研究: {state['topic'][:50]}..."],
        "next_agent": "writer"
    }

# 作家 Agent
def writer_agent(state: AgentState) -> AgentState:
    print("\n✍️ 作家 Agent 工作中...")
    
    prompt = ChatPromptTemplate.from_template(
        """你是一个专业的技术作家。基于研究员提供的笔记，撰写一篇结构清晰、通俗易懂的文章。

主题：{topic}

研究笔记：
{research_notes}

要求：
- 使用清晰的标题和段落结构
- 语言通俗易懂，适合初学者
- 包含实例说明
- 字数约500字

文章内容："""
    )
    
    chain = prompt | llm
    result = chain.invoke({
        "topic": state["topic"],
        "research_notes": state["research_notes"]
    })
    
    return {
        **state,
        "draft_content": result.content,
        "messages": [f"[作家] 完成初稿撰写"],
        "next_agent": "editor"
    }

# 编辑 Agent
def editor_agent(state: AgentState) -> AgentState:
    print("\n🔍 编辑 Agent 工作中...")
    
    prompt = ChatPromptTemplate.from_template(
        """你是一个专业的编辑。请审核并改进以下文章。

原文：
{draft_content}

请：
1. 检查语法和表达
2. 优化结构和逻辑
3. 确保内容准确性
4. 增强可读性

如果文章质量已经很好，保持原样并添加简短评价。
如果需要改进，请直接输出改进后的版本。

最终文章："""
    )
    
    chain = prompt | llm
    result = chain.invoke({"draft_content": state["draft_content"]})
    
    new_iteration = state["iteration"] + 1
    
    # 决定是否需要再次迭代
    if new_iteration >= state["max_iterations"]:
        next_agent = "END"
    else:
        # 可以根据质量评估决定是否继续
        next_agent = "END"  # 简化版本，一次迭代后结束
    
    return {
        **state,
        "final_content": result.content,
        "messages": [f"[编辑] 完成审核和改进"],
        "next_agent": next_agent,
        "iteration": new_iteration
    }

print("✅ Agent 定义完成！")


In [ ]:
from langgraph.graph import StateGraph, END

# 路由函数：决定下一个节点
def route_agent(state: AgentState) -> str:
    next_agent = state.get("next_agent", "researcher")
    if next_agent == "END":
        return END
    return next_agent

# 创建图
workflow = StateGraph(AgentState)

# 添加节点
workflow.add_node("researcher", researcher_agent)
workflow.add_node("writer", writer_agent)
workflow.add_node("editor", editor_agent)

# 设置入口
workflow.set_entry_point("researcher")

# 添加条件边
workflow.add_conditional_edges(
    "researcher",
    route_agent,
    {"writer": "writer", END: END}
)

workflow.add_conditional_edges(
    "writer",
    route_agent,
    {"editor": "editor", END: END}
)

workflow.add_conditional_edges(
    "editor",
    route_agent,
    {"writer": "writer", END: END}
)

# 编译
app = workflow.compile()

print("✅ 工作流构建完成！")


### 2.4 运行多 Agent 系统


In [ ]:
# 初始化状态
initial_state = {
    "topic": "深度学习中的 Transformer 架构",
    "research_notes": "",
    "draft_content": "",
    "final_content": "",
    "messages": [],
    "next_agent": "researcher",
    "iteration": 0,
    "max_iterations": 2
}

print("🚀 启动多 Agent 内容创作系统...")
print(f"📝 主题: {initial_state['topic']}\n")
print("="*60)

# 运行工作流
final_state = app.invoke(initial_state)

print("\n" + "="*60)
print("\n🎉 创作完成！")
print("\n📋 工作流程:")
for msg in final_state["messages"]:
    print(f"  {msg}")

print("\n" + "="*60)
print("\n📚 研究笔记:")
print(final_state["research_notes"][:300] + "...")

print("\n" + "="*60)
print("\n📄 最终文章:")
print(final_state["final_content"])


## 三、高级特性：动态路由和循环

### 3.1 带质量检查的循环改进


In [ ]:
# 质量评估 Agent
def quality_checker(state: AgentState) -> AgentState:
    print("\n🎯 质量检查 Agent 工作中...")
    
    prompt = ChatPromptTemplate.from_template(
        """评估以下文章的质量，并给出1-10的分数。

文章：
{content}

请只回答一个数字（1-10）和简短评价（一句话）。
格式：分数: X, 评价: XXX
"""
    )
    
    chain = prompt | llm
    result = chain.invoke({"content": state["final_content"]})
    
    # 简单解析分数
    try:
        score_str = result.content.split("分数:")[1].split(",")[0].strip()
        score = int(score_str)
    except:
        score = 8  # 默认分数
    
    # 如果分数低于8且未超过最大迭代次数，返回给作家改进
    new_iteration = state["iteration"] + 1
    if score < 8 and new_iteration < state["max_iterations"]:
        next_agent = "writer"
        message = f"[质检] 分数{score}/10, 需要改进"
    else:
        next_agent = "END"
        message = f"[质检] 分数{score}/10, 质量达标"
    
    return {
        **state,
        "messages": [message],
        "next_agent": next_agent,
        "iteration": new_iteration
    }

print("✅ 质量检查 Agent 定义完成！")


## 四、可视化工作流


In [ ]:
# 打印工作流结构
def visualize_workflow():
    print("\n📊 多 Agent 工作流结构:")
    print("\n")
    print("    ┌─────────────┐")
    print("    │   开始      │")
    print("    └──────┬──────┘")
    print("           │")
    print("           ↓")
    print("    ┌─────────────┐")
    print("    │  研究员      │ ← 收集信息")
    print("    └──────┬──────┘")
    print("           │")
    print("           ↓")
    print("    ┌─────────────┐")
    print("    │   作家       │ ← 撰写内容")
    print("    └──────┬──────┘")
    print("           │")
    print("           ↓")
    print("    ┌─────────────┐")
    print("    │   编辑       │ ← 审核改进")
    print("    └──────┬──────┘")
    print("           │")
    print("           ↓")
    print("    ┌─────────────┐")
    print("    │质量检查(可选)│ ← 评估质量")
    print("    └──────┬──────┘")
    print("           │")
    print("      [分数>=8?]")
    print("       ↙     ↘")
    print("     是       否")
    print("     ↓         ↓")
    print("   结束    返回作家")
    print("\n")

visualize_workflow()


## 五、更多实用案例

### 5.1 客服团队 Agent


In [ ]:
# 客服团队示例：分类 → 专家 → 质检

class CustomerServiceState(TypedDict):
    query: str
    category: str
    response: str
    satisfaction: str

def classifier_agent(state):
    """分类 Agent：判断问题类型"""
    print("📋 分类 Agent: 分析问题类型...")
    categories = ["技术问题", "账单问题", "产品咨询", "投诉建议"]
    # 简化：直接分类
    return {**state, "category": "技术问题"}

def technical_agent(state):
    """技术支持 Agent"""
    print("🔧 技术支持 Agent: 处理技术问题...")
    return {**state, "response": "这是技术支持的回答..."}

def billing_agent(state):
    """账单 Agent"""
    print("💰 账单 Agent: 处理账单问题...")
    return {**state, "response": "这是账单部门的回答..."}

print("✅ 客服团队 Agent 定义完成！")
print("\n💡 提示: 这个例子展示了如何根据分类路由到不同的专家 Agent")


### 5.2 代码审查团队


In [ ]:
# 代码审查团队：安全审查 + 性能审查 + 风格审查

class CodeReviewState(TypedDict):
    code: str
    security_issues: list
    performance_issues: list
    style_issues: list
    approved: bool

def security_reviewer(state):
    """安全审查 Agent"""
    print("🔒 安全审查 Agent: 检查安全漏洞...")
    # 这里可以集成实际的安全检查工具
    return {**state, "security_issues": []}

def performance_reviewer(state):
    """性能审查 Agent"""
    print("⚡ 性能审查 Agent: 分析性能问题...")
    return {**state, "performance_issues": []}

def style_reviewer(state):
    """代码风格 Agent"""
    print("🎨 风格审查 Agent: 检查代码规范...")
    return {**state, "style_issues": []}

print("✅ 代码审查团队定义完成！")
print("\n💡 提示: 可以并行运行多个审查 Agent，提高效率")


## 六、总结

### ✅ 掌握的技能

1. **LangGraph 基础**
   - State 状态管理
   - Node 节点定义
   - Edge 边和路由
   - Graph 工作流编排

2. **多 Agent 协作**
   - Agent 角色分工
   - 状态传递和共享
   - 条件路由
   - 循环和迭代

3. **实战模式**
   - 串行协作（研究→写作→编辑）
   - 并行处理（多个审查 Agent）
   - 条件分支（根据分类路由）
   - 质量循环（不达标重做）

### 🚀 应用场景

- 📝 内容创作团队
- 🎯 客户服务系统
- 🔍 代码审查流程
- 📊 数据分析管道
- 🤖 智能助手系统

### 💡 最佳实践

1. **清晰的状态设计**：定义完整的状态结构
2. **单一职责**：每个 Agent 专注一个任务
3. **错误处理**：考虑异常情况和回退机制
4. **性能优化**：并行处理独立任务
5. **可观测性**：添加日志和监控

---

**🎉 恭喜！你已经掌握了 LangGraph 多 Agent 系统的核心技能！**
